# 02. 모델 학습: 미래 예측하기 (LSTM) 🔮

> **목표**: 지난 10일간의 주가 흐름을 보고, **"내일 오를까?"**를 맞히는 인공지능을 만듭니다. 시계열(시간 순서가 있는) 데이터에 가장 강한 **LSTM** 모델을 사용합니다.

## 1. 데이터 준비 (재료 손질)
저장해둔 삼성전자 데이터를 불러옵니다. AI는 큰 숫자(80,000원)를 싫어하므로, **0과 1 사이로 압축(스케일링)**해줘야 합니다.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# 1. 데이터 불러오기
df = pd.read_csv("samsung_stock.csv", index_col='Date', parse_dates=True)
data = df[['Close']].values # '종가'만 사용

# 2. 스케일링 (0~1 사이로 변환)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

print("데이터 변환 완료:", scaled_data[:5])

## 2. 데이터셋 만들기 (문제집 만들기)
LSTM은 **"과거의 패턴"**을 보고 미래를 예측합니다.

**비유**:
- **문제**: [1일차, 2일차, ..., 10일차 가격]
- **정답**: [11일차 가격]

이렇게 10일 치를 묶어서 문제(X)로 주고, 다음 날을 정답(y)으로 주는 데이터셋을 만듭니다.

In [ ]:
def create_dataset(dataset, window_size=10):
    X, y = [], []
    for i in range(len(dataset) - window_size):
        X.append(dataset[i : i + window_size])
        y.append(dataset[i + window_size])
    return np.array(X), np.array(y)

window_size = 10
X, y = create_dataset(scaled_data, window_size)

# 훈련용/테스트용 나누기 (80% 훈련, 20% 테스트)
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

print("훈련 데이터 개수:", X_train.shape)

## 3. LSTM 모델 만들기 (뇌 구조 설계)
케라스(Keras)를 사용하면 블록 쌓듯이 쉽게 모델을 만들 수 있습니다.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

model = Sequential()
# 1. LSTM 층 (기억력 세포)
model.add(LSTM(50, return_sequences=False, input_shape=(window_size, 1)))
# 2. 출력 층 (예측값 내뱉기)
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
# 4. 학습 시작 (공부해!)
# epochs=20: 문제집을 20바퀴 반복해서 풀기
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

## 5. 결과 확인 및 저장
얼마나 잘 맞히는지 그래프로 그려보고, 잘 만든 모델은 파일(`models/my_stock_model.h5`)로 저장합니다.

In [ ]:
import matplotlib.pyplot as plt

# 예측하기
predictions = model.predict(X_test)

# 스케일링을 원래 가격으로 되돌리기
actual_prices = scaler.inverse_transform(y_test)
predicted_prices = scaler.inverse_transform(predictions)

# 그래프 그리기
plt.figure(figsize=(12, 6))
plt.plot(actual_prices, label='Actual Price', color='blue')
plt.plot(predicted_prices, label='Predicted Price', color='red')
plt.title('Samsung Electronics Price Prediction')
plt.legend()
plt.show()

# 모델 저장
model.save("models/my_stock_model.h5")
print("모델 저장 완료!")

## 6. ⚔️ 도전! 실습 미션
**미션: 학습 횟수(epochs)를 50으로 늘려서 다시 학습시켜보세요. 결과가 더 좋아지나요?**

In [ ]:
# 여기에 코드를 수정해서 작성해보세요
